In [1]:
import csv
import numpy as np  # http://www.numpy.org
import ast
from datetime import datetime
from math import log, floor, ceil

Modify the Utility class's methods. You can also add additional methods as required but don't change existing methods' arguments.

In [2]:
class Utility(object):
    
    # This method computes entropy for information gain
    def entropy(self, class_y):
        entropy = 0
        number_of_label = len(class_y) 
        if number_of_label <= 1:
            return 0
        
        value,counts = np.unique(class_y, return_counts = True)
      
        probs = counts/number_of_label        
        number_of_class = np.count_nonzero(probs)
        
        if number_of_class <= 1:
            return 0
        
        for i in probs:
            entropy -= i*log(i,2) 
        
        return entropy


    def partition_classes(self, X, y, split_attribute, split_val):
        X_left = []
        X_right = []
        y_left = []
        y_right = []
      
        for i in range(len(X)):
            x = X[i]
            if x[split_attribute] <= split_val:
                X_left.append(x)
                y_left.append(y[i])
            else:
                X_right.append(x)
                y_right.append(y[i])

#         print("X_left",*X_left,sep='\n')
#         print("X_right",*X_right,sep='\n')
#         print("y_left",y_left,sep='\n')
#         print("y_right",y_right,sep='\n')
        
        return (X_left, X_right, y_left, y_right)
    
    def information_gain(self, previous_y, current_y):
        # Inputs:
        #   previous_y: the distribution of original labels (0's and 1's)
        #   current_y:  the distribution of labels after splitting based on a particular
        #               split attribute and split value

        # TODO: Compute and return the information gain from partitioning the previous_y labels
        # into the current_y labels.
        # You will need to use the entropy function above to compute information gain
        # Reference: http://www.cs.cmu.edu/afs/cs.cmu.edu/academic/class/15381-s06/www/DTs.pdf

        """
        Example:

        previous_y = [0,0,0,1,1,1]
        current_y = [[0,0], [1,1,1,0]]

        info_gain = 0.45915
        """
        info_gain = self.entropy(previous_y)
        for i in current_y:
            info_gain -= self.entropy(i)*len(i)/len(previous_y)
        
        return info_gain


    def best_split(self, X, y):
        # Inputs:
        #   X       : Data containing all attributes
        #   y       : labels
        # TODO: For each node find the best split criteria and return the 
        # split attribute, spliting value along with 
        # X_left, X_right, y_left, y_right (using partition_classes)
        '''

        NOTE: Just like taught in class, don't use all the features for a node.
        Repeat the steps:

        1. Select m attributes out of d available attributes
        2. Pick the best variable/split-point among the m attributes
        3. return the split attributes, split point, left and right children nodes data 

        '''
        X_left, X_right, y_left, y_right = [], [], [], []

        split_attribute = 0
        split_value = 0
        current_info_gain = 0

        for index in range(0,len(X[0])):
            #create an array for all values for each attribute
            m = np.asarray(X)[:,index]
            split_val = np.asarray(np.mean([i for i in m]))

            X_left, X_right, y_left, y_right = self.partition_classes(X,y, index, split_val)

            info_gain = self.information_gain(y,[y_left, y_right])

            if info_gain > current_info_gain:
                current_info_gain = info_gain
                split_value = split_val
                split_attribute = index
        
        X_left, X_right, y_left, y_right = self.partition_classes(X, y, split_attribute, split_value)
        
#         print('current_info_gain:',current_info_gain)
#         print('split_value:',split_value)
#         print('split_attribute:',split_attribute)
#         print('X_left:',X_left)
#         print('X_right:',X_right)
#         print('y_left:',y_left)
#         print('y_right:',y_right)            
        return split_attribute,split_value,X_left, X_right,y_left,y_right

### Define the classes 'DecisionTree' and 'RandomForest'

Please modify the 'DecisionTree' and 'RandomForest' classes below

In [3]:
class DecisionTree(Utility):
    def __init__(self, max_depth):
        # Initializing the tree as an empty dictionary or list, as preferred
        self.tree = [-1,0,np.nan,np.nan] #set up tree structure: [split_attribute index, split_value, left, right]
        self.leaf = 3
        self.max_depth = max_depth
    
    def build_tree(self,X,y,depth =0):
        #set up termination condition
        if depth > self.max_depth or len(set(y)) == 1 or len(y) <= self.leaf:
            return [-1,np.mean(y),np.nan,np.nan]
        #create the node
        split_attribute, split_value, XLeft,XRight, yLeft, yRight= self.best_split(X, y)
        #call the recusive function to build the tree
        left = self.build_tree(XLeft,yLeft,depth +1)
        right = self.build_tree(XRight,yRight,depth +1)
        
#         print(split_attribute, split_value, left, right)
        return [split_attribute, split_value, left, right]
    
    def learn(self, X, y, depth=0):
        # TODO: Train the decision tree (self.tree) using the the sample X and labels y
        # You will have to make use of the functions in Utility class to train the tree

        # Use the function best_split in Utility class to get the best split and 
        # data corresponding to left and right child nodes
        
        # One possible way of implementing the tree:
        #    Each node in self.tree could be in the form of a dictionary:
        #       https://docs.python.org/2/library/stdtypes.html#mapping-types-dict
        #    For example, a non-leaf node with two children can have a 'left' key and  a 
        #    'right' key. You can add more keys which might help in classification
        #    (eg. split attribute and split value)
        ### Implement your code here
        #############################################
        self.tree = self.build_tree(X,y,0)
        #############################################
        
    def classify_temp(self, record,tree):

        split_attribute = tree[0]
        split_value = tree[1]
        left = tree[2]
        right = tree[3]
#         print(split_attribute, split_value, left, right)
        
        #the termininate condiction
        if split_attribute == -1:
            return split_value
        
        if record[split_attribute] <= split_value:
            return self.classify_temp(record,left)
        else:
            return self.classify_temp(record,right)

    def classify(self, record):
        # TODO: classify the record using self.tree and return the predicted label
        ### Implement your code here
        #############################################
        return self.classify_temp(record,self.tree)
        #############################################

In [4]:
class RandomForest(object):
    num_trees = 0
    decision_trees = []

    # the bootstrapping datasets for trees
    # bootstraps_datasets is a list of lists, where each list in bootstraps_datasets is a bootstrapped dataset.
    bootstraps_datasets = []

    # the true class labels, corresponding to records in the bootstrapping datasets
    # bootstraps_labels is a list of lists, where the 'i'th list contains the labels corresponding to records in
    # the 'i'th bootstrapped dataset.
    bootstraps_labels = []

    def __init__(self, num_trees):
        # Initialization done here
        np.random.seed(10)
        self.num_trees = num_trees
        self.decision_trees = [DecisionTree(max_depth=50) for i in range(num_trees)]
        self.bootstraps_datasets = []
        self.bootstraps_labels = []

    def _bootstrapping(self, XX, n):
        # Reference: https://en.wikipedia.org/wiki/Bootstrapping_(statistics)
        #
        # TODO: Create a sample dataset of size n by sampling with replacement
        #       from the original dataset XX.
        # Note that you would also need to record the corresponding class labels
        # for the sampled records for training purposes.

        samples = []  # sampled dataset
        labels = []  # class labels for the sampled records
        for i in range(n):
            index = np.random.randint(len(XX))
            samples.append(XX[index][:-1])
            labels.append(XX[index][-1])
        return (samples, labels)

    def bootstrapping(self, XX):
        # Initializing the bootstap datasets for each tree
        for i in range(self.num_trees):
            data_sample, data_label = self._bootstrapping(XX, len(XX))
            self.bootstraps_datasets.append(data_sample)
            self.bootstraps_labels.append(data_label)

    def fitting(self):
        # TODO: Train `num_trees` decision trees using the bootstraps datasets
        # and labels by calling the learn function from your DecisionTree class.
        for i in range(self.num_trees):
            X = self.bootstraps_datasets[i]
            y = self.bootstraps_labels[i]
            self.decision_trees[i].learn(X,y)

    def voting(self, X):
        y = []

        for record in X:
            # Following steps have been performed here:
            #   1. Find the set of trees that consider the record as an
            #      out-of-bag sample.
            #   2. Predict the label using each of the above found trees.
            #   3. Use majority vote to find the final label for this recod.
            votes = []
            for i in range(len(self.bootstraps_datasets)):
                dataset = self.bootstraps_datasets[i]
                if record not in dataset:
                    OOB_tree = self.decision_trees[i]
                    effective_vote = OOB_tree.classify(record)
                    votes.append(effective_vote)

            counts = np.bincount(votes)

            
            if len(counts) == 0:
                # TODO: Special case
                #  Handle the case where the record is not an out-of-bag sample
                #  for any of the trees.
                
                ind = -1
                for index in range(len(self.bootstraps_datasets)):
                    ind = self.bootstraps_datasets[index].index(record)
                    if ind != -1:
                        y = np.append(y, self.bootstraps_labels[index][ind])
                        break

            else:
                y = np.append(y, np.argmax(counts))

        return y
    
    def user(self):
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """
        ### Implement your code here
        #############################################
        return 'Zliu723'
        #############################################

In [5]:
# TODO: Initialize according to your implementation
# VERY IMPORTANT: Minimum forest_size should be 10
forest_size = 20

### Do not modify the below cell

In [6]:
# start time 
start = datetime.now()
X = list()
y = list()
XX = list()  # Contains data features and data labels
numerical_cols = set([i for i in range(0, 9)])  # indices of numeric attributes (columns)

# Loading data set
print("reading the data")
with open("pima-indians-diabetes.csv") as f:
    next(f, None)
    for line in csv.reader(f, delimiter=","):
        xline = []
        for i in range(len(line)):
            if i in numerical_cols:
                xline.append(ast.literal_eval(line[i]))
            else:
                xline.append(line[i])

        X.append(xline[:-1])
        y.append(xline[-1])
        XX.append(xline[:])

# Initializing a random forest.
randomForest = RandomForest(forest_size)

# printing the name
print("__Name: " + randomForest.user()+"__")

# Creating the bootstrapping datasets
print("creating the bootstrap datasets")
randomForest.bootstrapping(XX)

# Building trees in the forest
print("fitting the forest")
randomForest.fitting()

# Calculating an unbiased error estimation of the random forest
# based on out-of-bag (OOB) error estimate.
y_predicted = randomForest.voting(X)

# Comparing predicted and true labels
results = [prediction == truth for prediction, truth in zip(y_predicted, y)]

# Accuracy
accuracy = float(results.count(True)) / float(len(results))

print("accuracy: %.4f" % accuracy)
print("OOB estimate: %.4f" % (1 - accuracy))

# end time
print("Execution time: " + str(datetime.now() - start))

reading the data
__Name: Zliu723__
creating the bootstrap datasets
fitting the forest
accuracy: 0.7366
OOB estimate: 0.2634
Execution time: 0:00:07.125732
